In [20]:
import warnings
warnings.filterwarnings('ignore')
import unyt as u
import pickle
import numpy as np
import hoomd
import gsd
from flowermd.utils import get_target_box_mass_density

In [11]:
reference_values = {'energy': u.unyt_quantity(1.7782, 'kJ/mol'),
 'length': u.unyt_quantity(0.36, 'nm'),
 'mass': u.unyt_quantity(32.06, 'amu')}

In [12]:
rel_const_pos = np.load('assets/const_rel_pos.npy')
const_particle_types = ['ca', 'ca', 'ca', 'ca', 'sh', 'ca', 'ca']
with open("assets/pps_ff.pkl", "rb") as f:
    pps_ff = pickle.load(f)

In [40]:
# getting the box size with the desired density
N_beads = 800
pps_mass = u.unyt_quantity(110.174 , 'amu')

target_density = 0.95* u.Unit("g/cm**3")
system_mass = N_beads * pps_mass 

target_L = get_target_box_mass_density(density=target_density, mass=system_mass).to('nm')
target_L_reduced = target_L / reference_values['length']
target_L_reduced

unyt_array([14.89117676, 14.89117676, 14.89117676], '(dimensionless)')

### Rigid Body Simulation

In [41]:
def create_rigid_simulation(kT, init_snapshot, pps_ff, rel_const_pos, dt, tau, special_LJ=False, write_freq=1000):
    const_particle_types = ['ca', 'ca', 'ca', 'ca', 'sh', 'ca', 'ca']
    rigid_simulation = hoomd.Simulation(device=hoomd.device.auto_select(), seed=1)
    rigid_simulation.create_state_from_snapshot(init_snapshot)


    rigid = hoomd.md.constrain.Rigid()
    rigid.body['rigid'] = {
        "constituent_types":const_particle_types,
        "positions": rel_const_pos,
        "orientations": [(1.0, 0.0, 0.0, 0.0)]* len(rel_const_pos),
        }
    integrator = hoomd.md.Integrator(dt=dt, integrate_rotational_dof=True)
    rigid_simulation.operations.integrator = integrator
    integrator.rigid = rigid
    rigid_centers_and_free = hoomd.filter.Rigid(("center", "free"))
    nvt = hoomd.md.methods.ConstantVolume(
        filter=rigid_centers_and_free,
        thermostat=hoomd.md.methods.thermostats.MTTK(kT=kT, tau=tau))
    integrator.methods.append(nvt)
    
    cell = hoomd.md.nlist.Cell(buffer=0, exclusions=['body'])
    
    lj = hoomd.md.pair.LJ(nlist=cell)
    
    # use aa pps simulation to define lj and special lj forces between constituent particles
    for k, v in dict(pps_ff[0].params).items():
        lj.params[k] = v
        lj.r_cut[k] = 4.8
    
    lj.params[('rigid', ['rigid', 'ca', 'sh'])]= dict(epsilon=0, sigma=0)
    lj.r_cut[('rigid', ['rigid', 'ca', 'sh'])] = 0

    integrator.forces.append(lj)
    if special_LJ:
        special_lj = hoomd.md.special_pair.LJ()
        for k, v in dict(pps_ff[1].params).items():
            special_lj.params[k] = v
            special_lj.r_cut[k] = 4.8
        
        special_lj.params[('rigid', ['rigid', 'ca', 'sh'])]= dict(epsilon=0, sigma=0)
        special_lj.r_cut[('rigid', ['rigid', 'ca', 'sh'])] = 0

        integrator.forces.append(special_lj)
        
        
    rigid_simulation.state.thermalize_particle_momenta(filter=rigid_centers_and_free,
                                             kT=kT)
    
    rigid_simulation.run(0)

    
    log_quantities = [
                        "kinetic_temperature",
                        "potential_energy",
                        "kinetic_energy",
                        "volume",
                        "pressure",
                        "pressure_tensor",
                    ]
    logger = hoomd.logging.Logger(categories=["scalar", "string", "particle"])
    logger.add(rigid_simulation, quantities=["timestep", "tps"])
    thermo_props = hoomd.md.compute.ThermodynamicQuantities(filter=rigid_centers_and_free)
    rigid_simulation.operations.computes.append(thermo_props)
    logger.add(thermo_props, quantities=log_quantities)
    
    # for f in integrator.forces:
    #     logger.add(f, quantities=["energy", "forces", "energies"])

    logger.add(rigid_simulation.operations.integrator.rigid, quantities=["torques", "forces", "energies"])
    
    gsd_writer = hoomd.write.GSD(
        filename="trajectory.gsd",
        trigger=hoomd.trigger.Periodic(int(write_freq)),
        mode="wb",
        logger=logger,
        filter=hoomd.filter.All(),
        dynamic=["momentum", "property"]
        )
    
    rigid_simulation.operations.writers.append(gsd_writer)

    table_logger = hoomd.logging.Logger(categories=["scalar", "string"])
    table_logger.add(rigid_simulation, quantities=["timestep", "tps"])
    table_logger.add(thermo_props, quantities=log_quantities)
    table_file = hoomd.write.Table(
            output=open("log.txt", mode="w", newline="\n"),
            trigger=hoomd.trigger.Periodic(int(write_freq)),
            logger=table_logger,
            max_header_len=None,
        )
    rigid_simulation.operations.writers.append(table_file)
    return rigid_simulation

    
def shrink_volume(simulation, kT, target_box, n_steps):
    # box resizer
    final_box = hoomd.Box(
            Lx=target_box,
            Ly=target_box,
            Lz=target_box,
        )
    resize_trigger = hoomd.trigger.Periodic(100)
    box_ramp = hoomd.variant.Ramp(
        A=0, B=1, t_start=simulation.timestep, t_ramp=int(n_steps)
    )
    initial_box = simulation.state.box

    box_resizer = hoomd.update.BoxResize(
        box1=initial_box,
        box2=final_box,
        variant=box_ramp,
        trigger=resize_trigger,
    )
    simulation.operations.updaters.append(box_resizer)
    simulation.run(n_steps + 1, write_at_start=True)
    simulation.operations.updaters.remove(box_resizer)

    
    

In [42]:
init_snapshot = gsd.hoomd.open("assets/rigid_init_800.gsd")[-1]
rigid_sim = create_rigid_simulation(kT=0.1, init_snapshot=init_snapshot, pps_ff=pps_ff, rel_const_pos=rel_const_pos, 
                                    dt=0.0015, tau=0.1, special_LJ=False, write_freq=1e4)

In [43]:
shrink_volume(rigid_sim, kT=1.0, target_box=target_L_reduced[0], n_steps=1e5)

In [44]:
rigid_sim.state.get_snapshot().configuration.box

(14.89117675696174, 14.89117675696174, 14.89117675696174, 0.0, 0.0, 0.0)

In [45]:
rigid_centers_and_free = hoomd.filter.Rigid(("center", "free"))
rigid_sim.operations.integrator.methods = [hoomd.md.methods.ConstantVolume(
        filter=rigid_centers_and_free,
        thermostat=hoomd.md.methods.thermostats.MTTK(kT=3.0, tau=0.1))]


In [46]:
rigid_sim.run(40000)

In [47]:
hoomd.write.GSD.write(state=rigid_sim.state, mode='wb', filename='assets/rigid_after_shrink_{}_d_0.95.gsd'.format(N_beads))

In [48]:
for writer in rigid_sim.operations.writers:
    if hasattr(writer, 'flush'):
        writer.flush()

In [49]:
log = np.genfromtxt("log.txt", names=True)

potential_energy = log['mdcomputeThermodynamicQuantitiespotential_energy']

In [50]:
potential_energy / 800

array([ 12.529875  ,  -0.3300825 ,  -0.50223375,  -0.75178125,
        -1.04907   ,  -1.3997125 ,  -1.99195   ,  -2.9254875 ,
        -4.7913125 ,  -8.1028    , -14.661     , -12.758     ,
       -12.639125  , -12.650375  , -12.67875   ])

In [52]:
with rigid_sim._state.cpu_local_snapshot as snap:
    rtag = snap.particles.rtag[696]
    print(snap.particles.position[rtag])

HOOMDArray([6.98589389 2.04058073 7.20448373])
